## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll compute the camera calibration using chessboard images

In [6]:
# Set true if you want images to display

display = False

In [7]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('./camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        if display:
            # Draw and display the corners
            img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
            cv2.imshow('img',img)
            cv2.waitKey(500)

cv2.destroyAllWindows()

In [8]:
# Camera calibration parameters

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

In [9]:
# Test checkerboard undistortion

def cal_undistort(img, mtx, dist):
    # Use cv2.calibrateCamera() and cv2.undistort()
    undist = cv2.undistort(img, mtx, dist)
    return undist

if display:
    # Make a list of calibration images
    images = glob.glob('./camera_cal/calibration*.jpg')

    # Step through the list and search for chessboard corners
    for fname in images:
        img = cv2.imread(fname)
        undistorted = cal_undistort(img, mtx, dist)
        cv2.imshow('img',undistorted)
        cv2.waitKey(500)

    cv2.destroyAllWindows()


    # Read in an image
    img = cv2.imread('test_image.png')

In [10]:
# Check undistortion on test images of roads

if display:
    images = glob.glob('./test_images/*.jpg')

    for fname in images:
        img = cv2.imread(fname)
        undistorted = cal_undistort(img, mtx, dist)
        cv2.imshow('img', undistorted)
        cv2.waitKey(5000)

    cv2.destroyAllWindows()

In [257]:
# Helper functions

import matplotlib.image as mpimg
from numpy.linalg import inv

%matplotlib inline

def get_binary_image(img, binary='grey'):
    if binary == 'grey':
        return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    elif binary == 'HLS_S':
        return cv2.cvtColor(img, cv2.COLOR_BGR2HLS)[:,:,2]
    elif binary == 'HSV_V':
        return cv2.cvtColor(img, cv2.COLOR_BGR2HSV)[:,:,2]
    else:
        return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def abs_sobel_thresh(img, orient='x', thresh_min=0, thresh_max=255, binary='grey'):
    grey = get_binary_image(img, binary)
    
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(grey, cv2.CV_64F, 1, 0))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(grey, cv2.CV_64F, 0, 1))
        
    scaled_sobel = 255 * abs_sobel / np.max(abs_sobel)
    
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1
    return binary_output

def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255), binary='grey'):
    grey = get_binary_image(img, binary)
    
    sobel_x = np.absolute(cv2.Sobel(grey, cv2.CV_64F, 1, 0))
    sobel_y = np.absolute(cv2.Sobel(grey, cv2.CV_64F, 0, 1))
    sobel_xy = np.sqrt((sobel_x ** 2) + (sobel_y ** 2))
        
    scaled_sobel = 255 * sobel_xy / np.max(sobel_xy)
    
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= mag_thresh[0]) & (scaled_sobel <= mag_thresh[1])] = 1
    return binary_output

def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2), binary='grey'):
    grey = get_binary_image(img, binary)
    
    sobel_x = np.absolute(cv2.Sobel(grey, cv2.CV_64F, 1, 0))
    sobel_y = np.absolute(cv2.Sobel(grey, cv2.CV_64F, 0, 1))
        
    gradient_sobel = np.arctan2(sobel_y, sobel_x)
    
    binary_output = np.zeros_like(gradient_sobel)
    binary_output[(gradient_sobel >= thresh[0]) & (gradient_sobel <= thresh[1])] = 1
    return binary_output

def colour_threshold(img, thresh=(0, np.pi/2), binary='grey'):
    grey = get_binary_image(img, binary)
    
    binary_output = np.zeros_like(grey)
    binary_output[(grey >= thresh[0]) & (grey <= thresh[1])] = 1
    return binary_output

def window_mask(width, height, img_ref, center, level):
    output = np.zeros_like(img_ref)
    start_y = int(img_ref.shape[0] - (level + 1) * height)
    end_y = int(img_ref.shape[0] - level * height)
    start_x = max(0, int(center - width/2))
    end_x = min(int(center + width/2), img_ref.shape[1])
    
    output[start_y:end_y, start_x:end_x] = 1
    return output

def find_window_centroids(image, window_width, window_height, margin):
    window_centroids = [] # Store the (left,right) window centroid positions per level
    window = np.ones(window_width) # Create our window template that we will use for convolutions
    
    levels = int(image.shape[0] / window_height)
    
    # First find the two starting positions for the left and right lane by using np.sum to get the 
    # vertical image slice and then np.convolve the vertical image slice with the window template 
    
    # Sum quarter bottom of image to get slice, could use a different ratio
    
    y_start = int(3*image.shape[0]/4)
    x_mid = int(image.shape[1]/2)
    
    l_sum = np.sum(image[y_start:, :x_mid], axis=0)
    l_centre = np.argmax(np.convolve(window, l_sum)) - window_width/2
    r_sum = np.sum(image[y_start:, x_mid:], axis=0)
    r_centre = np.argmax(np.convolve(window, r_sum)) - window_width/2 + int(image.shape[1]/2)
    
    # Add what we found for the first layer
    
    window_centroids.append((l_centre, r_centre))
    
    # Go through each layer looking for max pixel locations
    
    for level in range(1, levels):
        # convolve the window into the vertical slice of the image
        y_level_start = int(image.shape[0] - (level + 1) * window_height)
        y_level_end = int(image.shape[0] - level * window_height)
        
        image_layer = np.sum(image[y_level_start:y_level_end, :], axis=0)
        conv_signal = np.convolve(window, image_layer)
        
        # Find the best left centroid by using past left center as a reference
        # Use window_width/2 as offset because convolution signal reference is 
        # at right side of window, not center of window
        
        offset = window_width / 2
        l_min_index = int(max(l_centre + offset - margin, 0))
        l_max_index = int(min(l_centre + offset + margin, image.shape[1]))

        # Update l_centre
        if np.max(conv_signal[l_min_index:l_max_index]) != 0:
            l_centre = np.argmax(conv_signal[l_min_index:l_max_index]) + l_min_index - offset
        
        # Find the best right centroid by using past right center as a reference
        
        r_min_index = int(max(r_centre + offset - margin, 0))
        r_max_index = int(min(r_centre + offset + margin, image.shape[1]))

        # Update r_centre
        if np.max(conv_signal[r_min_index:r_max_index]) != 0:
            r_centre = np.argmax(conv_signal[r_min_index:r_max_index]) + r_min_index - offset
        
        # Add what we found for that layer
        window_centroids.append((l_centre, r_centre))

    return window_centroids

In [348]:
# Process image function

def process_image(img):
    # x side offsets for lane projections 
    
    offset = 250

    # Coordinates of trapezoid for projection
    
    src = np.float32([(598, 446), (683, 446), (1064, 681), (241, 681)])
    
    # Coordinates of projected lane with x side offsets
    
    dst = np.float32([(offset, 0), (1280 - offset, 0), \
                      (1280 - offset, 720), (offset, 720)])

    # Given src and dst points, calculate the perspective transform matrix
    
    M = cv2.getPerspectiveTransform(src, dst)

    # Search window settings
    
    window_width = 50 
    window_height = 80 # Break image into 9 vertical layers since image height is 720
    margin = 50 # How much to slide left and right for searching

    undistorted = cal_undistort(img, mtx, dist)

    thresh_x = abs_sobel_thresh(undistorted, 'x', 5, 255, 'grey')
    thresh_y = abs_sobel_thresh(undistorted, 'y', 5, 255, 'grey')
    S_thresh = colour_threshold(undistorted, (150, 255), 'HLS_S')
    V_thresh = colour_threshold(undistorted, (150, 255), 'HSV_V')

    # Combined threshold 
    combined_thresh = np.uint8(((thresh_x == 1) & (thresh_y == 1)) | ((S_thresh == 1) & (V_thresh == 1)))
    
    img_size = (combined_thresh.shape[1], combined_thresh.shape[0])
    
    # Warp the image using OpenCV warpPerspective()
    
    warped = cv2.warpPerspective(combined_thresh, M, img_size, flags=cv2.INTER_LINEAR)

    window_centroids = find_window_centroids(warped, window_width, window_height, margin)

    # If we found any window centers
    
    if len(window_centroids) > 0:
        left_x = np.array([])
        right_x = np.array([])
        left_y = np.array([])
        right_y = np.array([])

        # Points used to draw all the left and right windows
        
        l_points = np.zeros_like(warped)
        r_points = np.zeros_like(warped)

        # Go through each level and draw the windows
        for level in range(0, len(window_centroids)):

            # Window_mask is a function to draw window areas

            l_mask = window_mask(window_width, window_height, warped, window_centroids[level][0], level)
            r_mask = window_mask(window_width, window_height, warped, window_centroids[level][1], level)

            # Add graphic points from window mask here to total pixels found 

            l_points[(l_points == 255) | ((l_mask == 1))] = 255
            r_points[(r_points == 255) | ((r_mask == 1))] = 255

            # Get indices of white points within each window
            
            window_points_l = np.where((l_points == 255) & ((l_mask == 1)))
            left_x = np.append(left_x, window_points_l[1])
            left_y = np.append(left_y, window_points_l[0])
            window_points_r = np.where((r_points == 255) & ((r_mask == 1)))
            right_x = np.append(right_x, window_points_r[1])
            right_y = np.append(right_y, window_points_r[0])

        ploty = np.linspace(0, 719, num=720)# to cover same y-range as image

        # Use these points to fit a second order polynomial to pixel positions 
        left_fit = np.polyfit(left_y, left_x, 2)
        left_fitx = left_fit[0] * ploty ** 2 + left_fit[1] * ploty + left_fit[2]
        right_fit = np.polyfit(right_y, right_x, 2)
        right_fitx = right_fit[0] * ploty ** 2 + right_fit[1] * ploty + right_fit[2]    

        # Define conversions in x and y from pixels space to meters
        
        ym_per_pix = 30/720 # meters per pixel in y dimension
        xm_per_pix = 3.7/720 # meters per pixel in x dimension

        # Get camera centre x value in warped image
        
        camera_centre_x = (640 - src[3][0]) / (src[2][0] - src[3][0])
        camera_centre_x_warped = camera_centre_x * (780) + offset # 720 is the number of pixels the lane fills at the bottom of frame
            
        base_left_x = left_fit[0] * img_size[1] ** 2 + left_fit[1] * img_size[1] + left_fit[2]
        base_right_x = right_fit[0] * img_size[1] ** 2 + right_fit[1] * img_size[1] + right_fit[2]    
        
        dist_from_centre = round((camera_centre_x_warped - 0.5 * (base_right_x + base_left_x)) * xm_per_pix, 2)
        
        # Fit new polynomials to x,y in world space
        
        left_fit_cr = np.polyfit(left_y * ym_per_pix, left_x * xm_per_pix, 2)
        right_fit_cr = np.polyfit(right_y * ym_per_pix, right_x * xm_per_pix, 2)
        
        # Calculate the new radii of curvature
        
        radius_left = (1 + (2 * left_fit_cr[0] * img_size[1] * ym_per_pix + left_fit_cr[1]) ** 2) ** 1.5 / \
            abs(2 * left_fit_cr[0])
        radius_right = (1 + (2 * right_fit_cr[0] * img_size[1] * ym_per_pix + right_fit_cr[1]) ** 2) ** 1.5 / \
            abs(2 * right_fit_cr[0])
            
        average_radius = 0.5 * (radius_left + radius_right)
        
        # Create an image to draw the lines on
        warp_zero = np.zeros_like(warped).astype(np.uint8)
        color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

        # Recast the x and y points into usable format for cv2.fillPoly()
        pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
        pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
        pts = np.hstack((pts_left, pts_right))

        # Draw the lane onto the warped blank image
        cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    
        # Warp the blank back to original image space using inverse perspective matrix (Minv)
        newwarp = cv2.warpPerspective(color_warp, inv(M), img_size) 
        # Combine the result with the original image
        result = cv2.addWeighted(undistorted, 1, newwarp, 0.3, 0)
        
        cv2.putText(result, "Radius of curvature: " + str(average_radius) + "m", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255))
        cv2.putText(result, "Vehicle is " + str(dist_from_centre) + " m right of centre", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255))
        
        return result
    else:
        return np.array(cv2.merge((warped,warped,warped)), np.uint8)

In [349]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [350]:
write_output = 'test_videos_output/test_02.mp4'

##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
#clip1 = VideoFileClip("./project_video.mp4").subclip(23,27)
clip1 = VideoFileClip("./project_video.mp4")

white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(write_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/test_02.mp4
[MoviePy] Writing video test_videos_output/test_02.mp4



 12%|█▏        | 152/1261 [00:36<04:19,  4.28it/s]


 24%|██▍       | 303/1261 [01:12<03:42,  4.30it/s]


 36%|███▌      | 454/1261 [01:48<03:12,  4.18it/s]


 48%|████▊     | 605/1261 [02:23<02:34,  4.23it/s]


...

 72%|███████▏  | 907/1261 [03:35<01:24,  4.19it/s]


 84%|████████▍ | 1057/1261 [04:11<00:48,  4.23it/s]


 96%|█████████▌| 1205/1261 [04:46<00:13,  4.28it/s]


100%|█████████▉| 1260/1261 [04:58<00:00,  4.30it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/test_02.mp4 

CPU times: user 7min 19s, sys: 1min 26s, total: 8min 45s
Wall time: 4min 59s


In [106]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(write_output))